In [7]:
import pandas as pd
import gc

# Cargar datos
sellin = pd.read_csv("../../datasets/sell-in.txt.gz", sep="\t")
productos = pd.read_csv("../../datasets/tb_productos.txt", sep="\t")
productos = productos.drop_duplicates(subset=["product_id"],keep="first")  # Eliminar duplicados por si acaso
stocks = pd.read_csv("../../datasets/tb_stocks.txt", sep="\t")

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
del sellin, productos, stocks

# Agrupar ventas por periodo, cliente y producto
dt = df.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "cat1":"first",
                                                                "cat2":"first",
                                                                "cat3":"first",
                                                                "brand":"first",
                                                                "sku_size":"first",
                                                                "stock_final":"first",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
# 1. Calcular primeros períodos

periodo_producto =  df.groupby(['periodo', 'product_id'])["tn"].sum().rename('periodo_producto')
nacimiento_producto = df.groupby('product_id')['periodo'].min().rename('nacimiento_producto')
#muerte_cliente = df.groupby('customer_id')['periodo'].max().rename('muerte_cliente')

# 2. Crear todas las combinaciones posibles
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()
periodos = df['periodo'].unique()

# 3. Crear matriz de combinaciones válidas usando broadcasting
idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
# 4 filtrar combinaciones periodo_producto
completo = completo.merge(periodo_producto, on=['periodo', 'product_id'], how='left')
completo = completo[completo['periodo_producto'].notna()]
# 5 Filtrar combinaciones nacimiento_producto
completo = completo.merge(nacimiento_producto, on='product_id', how='left')
completo = completo[completo['periodo'] >= completo['nacimiento_producto']]
completo = completo[completo['nacimiento_producto'] < 201910]

# 6. Filtrar combinaciones muerte_cliente
# completo = completo.merge(muerte_cliente, on='customer_id', how='left')
# completo = completo[completo['periodo'] <= completo['muerte_cliente']]


# 7. Unir con datos reales
df_completo = completo.merge(dt, how='left')
df_completo['tn'] = df_completo['tn'].fillna(0)

# Convertir periodo a datetime con día fijo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear columna periodo_target_dt desplazada 2 meses
df_completo['periodo_target_dt'] = df_completo['periodo_dt'] + pd.DateOffset(months=2)

# crear target si compro o no en el periodo
df_completo['compro'] = (df_completo['tn'] > 0).astype(int)

# DataFrame auxiliar con ventas futuras
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'compro']].copy()
ventas_futuras = ventas_futuras.rename(columns={
    'periodo_dt': 'periodo_target_dt',
    'compro': 'target'
})

# Hacemos el merge usando fechas datetime directamente
dt = pd.merge(
    df_completo,
    ventas_futuras,
    how='left',
    on=['periodo_target_dt', 'customer_id', 'product_id']
)

del completo, idx, periodo_producto, nacimiento_producto, df_completo
gc.collect()

dt.shape

(18581028, 19)

In [5]:
dt.head()

,product_id,customer_id,periodo,periodo_producto,nacimiento_producto,tn,cust_request_tn,cust_request_qty,cat1,cat2,cat3,brand,sku_size,stock_final,plan_precios_cuidados,periodo_dt,periodo_target_dt,compro,target
0,20524,10234,201701,6.48085,201701,0.05300,0.05300,2.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.0,2017-01-01,2017-03-01,1,1.0
1,20524,10234,201702,3.99755,201701,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-01,2017-04-01,0,0.0
2,20524,10234,201703,7.14711,201701,0.01514,0.01514,1.0,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.0,2017-03-01,2017-05-01,1,0.0
3,20524,10234,201704,6.82163,201701,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-01,2017-06-01,0,1.0
4,20524,10234,201705,9.25949,201701,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-01,2017-07-01,0,0.0


In [12]:
dt.to_parquet("../../datasets/df_target_class.parquet", index=False)